In [4]:
!gsutil -m cp "gs://quickdraw_dataset/full/simplified/cat.ndjson" .
!gsutil -m cp "gs://quickdraw_dataset/full/simplified/dog.ndjson" .

Copying gs://quickdraw_dataset/full/simplified/cat.ndjson...
- [1/1 files][ 73.1 MiB/ 73.1 MiB] 100% Done                                    
Operation completed over 1 objects/73.1 MiB.                                     
Copying gs://quickdraw_dataset/full/simplified/dog.ndjson...
\
Operation completed over 1 objects/87.1 MiB.                                     


In [ ]:
!gsutil -m cp -I gs://quickdraw_dataset/full/simplified/ < classes.txt